# Fixture Observation

#### This script creates a fixture observation for each match, combining the home and away teams statistics, as well as the prediction target - pre-match odds

In [ ]:
import boto3
import pandas as pd
import numpy as np
import json
from io import StringIO


s3 = boto3.resource('s3')
bucket_name = 'betfairex'


def read_csv_from_s3(bucket_name, object_key):
    response = s3.Object(bucket_name, object_key).get()

    s3_contents = response['Body'].read()
    df = pd.read_csv(StringIO(s3_contents.decode('utf-8')))

    return df

In [ ]:
bucket = s3.Bucket(bucket_name)

In [ ]:
all_keys = [obj.key for obj in bucket.objects.all()]

# keys are in the regular expression as follows:
# YYYY-MM-DD_[HOME_TEAM] v [AWAY_TEAM].csv

In [ ]:
all_keys[-1]

In [ ]:
LEAGUE_INFO = pd.read_csv('league_info.csv')
TEAM_INFO = pd.read_csv('team_info.csv')

def get_match_info(object_key):
    match_name = object_key[:-4]
    match_date_str = match_name.split('_')[0]
    fixture = match_name.split('_')[1]
    home_team = fixture.split(' v ')[0]
    away_team = fixture.split(' v ')[1]
    
    match_info = {
        'match_date': match_date_str,
        'home_team': home_team,
        'away_team': away_team,
    }    
    return match_info

def get_betfair_exchange_prematch_odds(bucket_name, object_key):
    match_info = get_match_info(object_key)
    
    odds_df = read_csv_from_s3(bucket_name, object_key)
    
    odds_df = odds_df.sort_values("timestamp", ascending=True)
    prematch_df = odds_df[~odds_df['inplay']]
    
    home_team_df = prematch_df[prematch_df['selection'] == match_info['home_team']]
    away_team_df = prematch_df[prematch_df['selection'] == match_info['away_team']]
    draw_df = prematch_df[prematch_df['selection'] == "Draw"]
    
    home_prematch_odds = home_team_df.tail(1)['odds'].iloc[0]
    away_prematch_odds = away_team_df.tail(1)['odds'].iloc[0]
    draw_prematch_odds = draw_df.tail(1)['odds'].iloc[0]
    
    prematch_odds = {
        'match_date': match_info['match_date'],
        match_info['home_team']: home_prematch_odds,
        match_info['away_team']: away_prematch_odds,
        'Draw': draw_prematch_odds
    }
    return prematch_odds    
    

In [ ]:
# Configuration constants

SERVER = "v3.football.api-sports.io"
API_KEY = "1f5008c4f33481203631d90c7a81c5e5"

from pandas.tseries.offsets import Day
import http.client
import datetime as dt

SERVER = "v3.football.api-sports.io"



conn = http.client.HTTPSConnection(SERVER)

headers = {
  'x-rapidapi-host': SERVER,
  'x-rapidapi-key': API_KEY
}


def get_team_stats(season, league_id, team_id, as_of_date):
    # as_of_date in format YYYY-MM-DD
    conn.request("GET", "/teams/statistics?season=%s&league=%s&team=%s&date=%s" % (season, league_id, team_id, as_of_date), headers=headers)
    res = conn.getresponse()
    data = res.read()

    team_stats_raw = data.decode("utf-8")
    team_stats = json.loads(team_stats_raw)["response"]
    
    return team_stats
    

def get_team_stats_for_fixture(object_key):
    match_info = get_match_info(object_key)
    
    match_date = match_info['match_date']
    as_of_date = dt.datetime.strptime(match_date, "%Y-%m-%d") - Day(1)
    as_of_date = as_of_date.strftime("%Y-%m-%d")
    
    season = match_date[:4]
    
    home_team = match_info['home_team']
    home_team_info = TEAM_INFO.query("name == @home_team").to_dict('records')[0]
    home_team_id = home_team_info['team_id']
    home_team_league_id = home_team_info['league_id']
    home_team_stats = get_team_stats(season, home_team_league_id, home_team_id, as_of_date)
    
    away_team = match_info['away_team']
    away_team_info = TEAM_INFO.query("name == @away_team").to_dict('records')[0]
    away_team_id = away_team_info['team_id']
    away_team_league_id = away_team_info['league_id']
    away_team_stats = get_team_stats(season, away_team_league_id, away_team_id, as_of_date)
    
    return home_team_stats, away_team_stats
    

In [ ]:
home_team_stats, away_team_stats = get_team_stats_for_fixture(all_keys[-1])

In [ ]:
home_team_stats

In [ ]:
away_team_stats